In [1]:
import sys
sys.executable

'/home/rs37890/.conda/envs/diffusion/bin/python'

In [2]:
sys.path.append('/scratch/rs37890/CARC/Medical_images_source_MR/Deeplabv3_source_MR/')
sys.path.append('/scratch/rs37890/CARC/Medical_images_source_MR/Deeplabv3_source_MR/Adapt_1298/')

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms


import os
import sys
import h5py
import time
import numpy as np
import pandas as pd
from PIL import Image
from IPython.display import display as ipy_display, clear_output
import matplotlib.pyplot as plt

In [4]:
# setting path
deplabv3 = __import__('Deeplabv3')

In [5]:
# setting path
import networks

In [6]:
source_dataset =  __import__('step1_source_dataset')

In [7]:
target_dataset =  __import__('step1_target_dataset')

# parameters

In [8]:
num_classes = 5
batch_size = 16
suffix = 'run4'
epoch = 20000
dataset_name = 'abdomen'

# CUDA

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# Path

In [10]:
source_ct_train_dir = "/scratch/rs37890/CARC/Medical_images_source_MR/data/data/h5py/"
source_ct_test_dir = "/scratch/rs37890/CARC/Medical_images_source_MR/data/data/h5py/"

In [11]:
target_mr_train_dir = "/scratch/rs37890/CARC/Medical_images_source_MR/data/data/h5py/"
target_mr_test_dir = "/scratch/rs37890/CARC/Medical_images_source_MR/data/data/h5py/"

# label_ids_abdomen

In [12]:
label_ids_abdomen = {"ignore": 0,
    "lv_myo": 1,
    "la_blood": 2,
    "lv_blood": 3,
    "aa": 4,
}
label_ids = label_ids_abdomen

# Load model 

In [13]:
dpv3 = deplabv3.DeepLabV3(num_classes)
classifier = networks.classifier(num_classes)

dpv3 = dpv3.to(device)
classifier = classifier.to(device)

# parallel
dpv3 = torch.nn.DataParallel(dpv3)
classifier = torch.nn.DataParallel(classifier)

In [14]:
root = '/scratch/rs37890/CARC/Medical_images_source_MR/Deeplabv3_source_MR/record-data/'

In [15]:
dpv3_checkpoint = torch.load( root + f'dpv3_weights_{epoch}.pth')
classifier_checkpoint = torch.load( root + f'classifier_weights_{epoch}.pth')

dpv3.load_state_dict(dpv3_checkpoint)
classifier.load_state_dict(classifier_checkpoint)
print("Loaded model weights")

Loaded model weights


In [16]:
class Combined_Model(nn.Module):
    
    def __init__(self, Unet, classifier):
        super(Combined_Model, self).__init__()
        
        self.Unet = Unet
        self.classifier = classifier
        
    def forward(self, x):
        
        output1 = self.Unet(x)
        output2 = self.classifier(output1)
        
        return output2

In [17]:
# Create the combined model
combined = Combined_Model(dpv3, classifier)
combined = combined.to(device)

# Dataset

In [18]:
dataloader = source_dataset.get_dataloader( source_ct_train_dir,  source_ct_test_dir, \
                                           num_classes, batch_size, domain = 'source' )

source_train_dataset = dataloader["train"].dataset
source_test_dataset = dataloader["test"].dataset

In [19]:
dataloader = target_dataset.get_dataloader( target_mr_train_dir,  target_mr_test_dir, num_classes, \
                                           batch_size, domain = 'target' )

target_train_dataset = dataloader["train"].dataset
target_test_dataset = dataloader["test"].dataset

# learn gaussians

In [20]:
def softmax(x):
    e_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return e_x / np.sum(e_x, axis=-1, keepdims=True)

In [21]:
def learn_gaussians(data, y_true, z_model, sm_model, batch_size, label_ids, rho=.97, initial_means=None):    
    
    # z_model: deeplab v3
    # sm_model: combined
    
    num_classes = len(label_ids)
    
    means = initial_means
    if initial_means is None:
        means = np.zeros((num_classes, num_classes)) # (5,5)
        
    covs = np.zeros((num_classes, num_classes, num_classes)) # shapeL (5, 5, 5)
    cnt = np.zeros(num_classes) # shape: (5,)
    
    
    data = data.transpose((2, 0, 1)) # Before is [256, 256, 32] after is [32, 256, 256]
    data = np.expand_dims(data, axis=0) # [1, 32, 256, 256]
    data = np.expand_dims(data, axis=0) # [1, 1, 32, 256, 256]
    
    y_true = y_true.transpose((2, 0, 1)) # Before is [256, 256, 32] after is [32, 256, 256]
    y_true = np.expand_dims(y_true, axis=0) # [1, 32, 256, 256]
    
    
    assert len(data.shape) == 5
    assert len(y_true.shape) == 4
    
    assert data.shape[2] == 32
    assert y_true.shape[1] == 32
    
    num_of_correct_prediction_each_classes = np.zeros(5, dtype=int)
    
    z_model.eval()
    sm_model.eval()

    with torch.no_grad():

        input_data = torch.from_numpy(data).to(device) # input_data:  torch.Size([1, 1, 32, 256, 256]) 
        input_data = input_data.float()
        
        # Predict latent features
        zs = z_model(input_data) # (8, 5, 32, 256, 256)
        zs = zs.cpu()
        zs = zs.permute(0, 2, 3, 4, 1)
        zs = zs.reshape(-1, num_classes).detach().numpy() # (8 * 32 * 256 * 256, 5)


        # Get softmax outputs
        # (48, 5, 32, 256, 256) => (32 * 256 * 256 * 8, 5)
        y_hat = sm_model(input_data).softmax(dim = 1).cpu()
        y_hat = y_hat.permute(0, 2, 3, 4, 1)
        y_hat = y_hat.reshape(-1, num_classes).detach().numpy()
            
        # downwards across rows (axis 0) 
        # running horizontally across columns (axis 1) = (axis=-1)
        # Because of continual slice, the vmax likes  [0.2908752  0.29087755 0.29087812 ... 0.29117948 0.2911795  0.29118514]
        vmax = np.max(y_hat, axis = 1) 
        y_hat = np.argmax(y_hat, axis = 1) 
        
        # before ravel: (8, 32, 256, 256);
        y_t = y_true.ravel()
        
        # Keep a few exemplars per class
        for label in label_ids:
            c = label_ids[label]

            ind = (y_t == c) & (y_hat == c) & (vmax > rho)
            
            if np.sum(ind) > 0:
                # We have at least one sample
                curr_data = zs[ind]
                num_of_correct_prediction_each_classes[c] = np.sum(ind)

                if initial_means is None:
                    # Only update means and counts

                    # 1d + 1d == (5, ) + (5, )
                    means[c] += np.sum(curr_data, axis=0)
                    cnt[c] += np.sum(ind)

                else:
                    # ! here means are scaled to their final values
                    # Example: 
                    # curr_data.shape = (8, 5); 8 is the sum of ind
                    # means[c].shape:  (5,)
                    # curr_data - means[c]:  (8, 5)
                    # np.transpose(curr_data - means[c]:  (5, 8)
                    # np.dot = (5, 8) * (8, 5) = (5, 5)

                    sigma = np.dot(np.transpose(curr_data - means[c]), curr_data - means[c])
                    assert sigma.shape == (num_classes, num_classes)
                    covs[c] += sigma
                    cnt[c] += np.sum(ind)
        

    
    # Normalize results
    for i in range(num_classes):
        if initial_means is None:
            means[i] /= (cnt[i] + 1e-10)
        covs[i] /= (cnt[i] - 1)
        
    assert np.isnan(means).any() == False
    assert np.isnan(cnt).any() == False
    
    return means, covs, cnt, num_of_correct_prediction_each_classes

# Find the most similarity based on label space

In [22]:
filename = 'Gaussian_similarity_dataframe_1298.csv'

if os.path.exists(filename):
    
    df = pd.read_csv(filename)
    restart_image_idx = df.iloc[-1]['source_images_idx']
    restart_i = df.iloc[-1]['source_i']
    restart_j = df.iloc[-1]['source_j']
    restart_k = df.iloc[-1]['source_k']
    
else:
    df = pd.DataFrame(columns=["source_images_idx", "source_i", "source_j", "source_k", 
                               "means", "covs", "label_frequency", "perdict_frequency"])
    restart_image_idx = 0
    restart_i = 0
    restart_j = 0
    restart_k = 0

In [23]:
counter = 0

for source_idx in range( restart_image_idx, len(source_train_dataset) ):
    
    source_data, source_label = source_train_dataset[source_idx] 
    shape_source = source_label.shape # (126, 113, 203)
    crop_size = (32, 32, 32)

    total_steps_i = shape_source[0] - crop_size[0] + 1
    step_size_i = total_steps_i // 3  # Integer division to get whole number steps

    total_steps_j = shape_source[1] - crop_size[1] + 1
    step_size_j = total_steps_j // 3  # Integer division to get whole number steps

    total_steps_z = shape_source[2] - crop_size[2] + 1
    step_size_z = total_steps_z // 3

    # Slide the smaller array across the larger one
    if step_size_i != 0:
        x_range = [i for i in range( 0,  total_steps_i, step_size_i)]
        if x_range[-1] != (shape_source[0] - crop_size[0]):
            x_range.append(shape_source[0] - crop_size[0])
    else:
        x_range = [i for i in range( 0,  total_steps_i, 1)]
    
    if step_size_j != 0:
        y_range = [j for j in range( 0,  total_steps_j, step_size_j)]
        if y_range[-1] != (shape_source[1] - crop_size[1]):
            y_range.append(shape_source[1] - crop_size[1])
    else:
        y_range = [j for j in range( 0,  total_steps_j, 1)]
    
    if step_size_z != 0:
        z_range = [z for z in range( 0,  total_steps_z, step_size_z)]
        if z_range[-1] != (shape_source[2] - crop_size[2]):
            z_range.append(shape_source[2] - crop_size[2])
    else:
        z_range = [z for z in range( 0,  total_steps_z, 1)]
    
    # Slide the smaller array across the larger one
    for i in x_range:
        
        if i < restart_i: continue
        
        for j in y_range:
            
            if j < restart_j: continue
            
            for k in z_range:
                
                if k < restart_k: continue

                print(source_idx, i, j, k)
                print(len(df))
                counter += 1
                # Extract  
                data = source_data[i:i+crop_size[0], j:j+crop_size[1], k:k+crop_size[2]] # (32, 32, 32)
                y_true = source_label[i:i+crop_size[0], j:j+crop_size[1], k:k+crop_size[2]]
                
                means, _ , ct, _ = learn_gaussians(data, y_true, dpv3,\
                                                combined, batch_size, label_ids, rho=0.97, initial_means=None)
                

                means, covs, ct, y_perdict = learn_gaussians(data, y_true, dpv3,\
                                                  combined, batch_size, label_ids, rho=0.97, initial_means=means)
                
                
                
                n_samples = np.zeros(5, dtype=int)
                cls, ns = np.unique(y_true, return_counts=True)
                for h in range(len(cls)):
                    n_samples[int(cls[h])] = ns[h]
                
                n_samples_2 = y_perdict
                
                df = df.append({"source_images_idx": source_idx,
                                "source_i": i,
                                "source_j": j,
                                "source_k": k,
                                "means": means,
                                "covs": covs,
                                "label_frequency": n_samples, 
                                "perdict_frequency": n_samples_2}, ignore_index=True)
                
                if counter % 10000 == 0:
                    # Save DataFrame to .csv file
                    df.to_csv('Gaussian_similarity_dataframe_1298.csv', index=False)
                
                    # Clear last display and display the DataFrame
                    clear_output(wait=True)
                    display(df)
                    
                if restart_k !=0: restart_k = 0
            if restart_j != 0: restart_j = 0
        if restart_i != 0: restart_i = 0
                
                
df.to_csv('Gaussian_similarity_dataframe_1298.csv', index=False)              

0 0 0 0
0


/home/rs37890/.conda/envs/diffusion/lib/python3.9/site-packages/torch/nn/modules/conv.py:605: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1702400441250/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv3d(
/tmp/ipykernel_3842719/849445217.py:76: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"source_images_idx": source_idx,
/tmp/ipykernel_3842719/849445217.py:76: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"source_images_idx": source_idx,


0 0 0 107
1
0 0 0 214
2
0 0 0 321
3
0 0 0 322
4
0 0 4 0
5


/tmp/ipykernel_3842719/849445217.py:76: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"source_images_idx": source_idx,
/tmp/ipykernel_3842719/849445217.py:76: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"source_images_idx": source_idx,
/tmp/ipykernel_3842719/849445217.py:76: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"source_images_idx": source_idx,
/tmp/ipykernel_3842719/849445217.py:76: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"source_images_idx": source_idx,
/tmp/ipykernel_3842719/849445217.py:76: FutureWarning: The frame.append method is deprecated and will be

0 0 4 107
6
0 0 4 214
7
0 0 4 321
8
0 0 4 322
9
0 0 8 0
10


...

15 75 21 39
1359
15 75 21 78
1360
15 75 21 116
1361
15 75 42 0
1362
15 75 42 39
1363


/tmp/ipykernel_3842719/849445217.py:76: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"source_images_idx": source_idx,
/tmp/ipykernel_3842719/849445217.py:76: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"source_images_idx": source_idx,
/tmp/ipykernel_3842719/849445217.py:76: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"source_images_idx": source_idx,
/tmp/ipykernel_3842719/849445217.py:76: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"source_images_idx": source_idx,
/tmp/ipykernel_3842719/849445217.py:76: FutureWarning: The frame.append method is deprecated and will be

15 75 42 78
1364
15 75 42 116
1365
15 75 62 0
1366
15 75 62 39
1367
15 75 62 78
1368


/tmp/ipykernel_3842719/849445217.py:76: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"source_images_idx": source_idx,
/tmp/ipykernel_3842719/849445217.py:76: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"source_images_idx": source_idx,


15 75 62 116
1369


In [24]:
df.head()

,source_images_idx,source_i,source_j,source_k,means,covs,label_frequency,perdict_frequency
0,0,0,0,0,"[[8.081975754735533, -12.013315163772596, -13....","[[[3.886720538192222, -1.7020385284628534, -1....","[19268, 7577, 0, 5923, 0]","[12670, 313, 0, 667, 0]"
1,0,0,0,107,"[[8.474080787285642, -12.78317694036674, -14.6...","[[[5.772613142762121, -4.678539746130251, -5.3...","[29145, 3428, 0, 195, 0]","[25137, 487, 0, 0, 0]"
2,0,0,0,214,"[[7.4278178215026625, -8.546494483947727, -10....","[[[3.9643140175364366, -4.376897973424478, -6....","[32768, 0, 0, 0, 0]","[32768, 0, 0, 0, 0]"
3,0,0,0,321,"[[6.37268877029417, -6.095223903655987, -8.389...","[[[2.019889040766233, -0.4134280617820215, -1....","[32768, 0, 0, 0, 0]","[32768, 0, 0, 0, 0]"
4,0,0,0,322,"[[6.3714575767516894, -6.075657844543438, -8.3...","[[[2.0379151898599437, -0.3537983919287831, -1...","[32768, 0, 0, 0, 0]","[32768, 0, 0, 0, 0]"


In [25]:
ns

array([32768])

In [26]:
y_perdict

array([32768,     0,     0,     0,     0])

In [27]:
len(df)
# 945773

1370